In this lesson we will show a data structure that comes up quite often in competitive programming. The $segment tree$'s main purpose is to solve problems (or subproblems) that receive a flow of $queries$ and $updates$ on an array and need to solve the queries online. Let's take an example:
You are given an array $a$ of $N$ integers, numbered $1$ through $N$, and $M$ operations, each one either:
- $update$: given $x$ and $y$, change $a[x]$ to $y$
- $query$: given $x$ and $y$, find $max(a[x], a[x+1]...a[y])$
Give the answer to each query.

#### Straight forward solution
For the rest of the lesson, we will denote a subarray of the given array as a $segment$.
The first idea that comes to mind when solving this problem is to go through all elements in the segment of the query and just find the maximum value among them in $O(y-x)=O(N)$ while at an update just change $a[x]$ to $y$, $O(1)$ time.
    
#### Optimization idea

For simplicity of explanation, let's assume $N$ is a power of $2$, so $N=2^k$. 
Now, you can think of the solution above in this way: we have the calculated solution for every segment of length $1$ (the maximum value of segment $i..i$ is $a[i]$). They are updated in $O(1)$, but a query is formed from $O(N)$ such segments. What if we were to have the solution calculated for a few /more/ segments, inteligently maintained, so that even though maybe you would have to recalculate more than one segment in an update, the number of segments needed for a query would decrease significantly?
Great, but now what segments should we maintain?
We will maintain a total of $2N-1$ segments: one of length $N$, two of length $N/2$, $4$ of length $N/4$ and so on, and their structure will look like this:
    
    
Every segment of length $2^k$ will split into $2$ segments of length $2^{k-1}$, which from now on we will call its $sons$. The segment itself will be the $father$ of these two segments.
Of course, now comes the idea to maintain the segments in a <Definition term="tree$-like structure.

#### Segment trees
A segment tree has the structure of a full binary tree, so it can be stocked in the memory in the same way as a  heap, as a simple array, where the root of the tree is $0$, the father of a segment $i$ is $\left \lfloor{(i-1)/2}\right \rfloor$ and its two sons are $2i+1$ and $2i+2$. Here is the representation of the segments above in tree fashion:
    
If $N$ is not a power of $2$, the only thing that changes is that if you have an odd-lengthed segment, the left side will be of length $\left \lceil{L/2}\right \rceil$ while the right side will be of length $\left \lfloor{L/2}\right \rfloor$. The tree will no longer be /full/, but all other properties still hold. You can check out how it looks like by clicking on the `custom input` button.

#### Building the tree
For the segment tree to work, you need to be able to calculate the /solution/ for a segment from the solutions of its sons. For the sample problem, the maximum value on a segment is the maximum between the answers in its two sons, so the calculation looks like this:

In [1]:
void recalculate(int node) {
    //calculate the solution for the current segment,
    //considering the sons are correctly solved
    segm_tree[node] = max(segm_tree[2 * node + 1], segm_tree[2 * node + 2]);
}


SyntaxError: invalid syntax (<ipython-input-1-1714e8aee903>, line 1)

Building the tree for the initial array will be done in a recursive way, as each segment will be able to calculate its solution from the solutions stored in its two sons. This function looks the same for almost any info stored in a segment tree:

In [2]:
void build(int node, int left, int right) { //"node" is the index in the array, while "left"
                                            // and "right" are the ends of the current segment
    if (left == right) {
        segm_tree[node] = a[left]; //we are in a leaf node
    } else {
        int middle = (left + right) / 2;
        build(2 * node + 1, left, middle);
        build(2 * node + 2, middle + 1, right);
        recalculate(node);
    }
}

...
build(0, 1, N);
...

SyntaxError: invalid syntax (<ipython-input-2-58e38b08f15f>, line 1)

Since the <Latex value="N"> (length of the array) does not change during the run-time (if it does, you need a different data structure like <Link href="https://en.wikipedia.org/wiki/Treap" value="treaps">), each node of the tree (basically each index in the `segm_tree` array) will store the solution for the same segment at all times, so with the implementations we will use, you don't need to store the ends of the current segment, just pass them along as parameters to all functions.
This function goes through every node of the segment tree exactly once, so its time complexity is <Latex value="O(N)">.

#4 Update
At an operation of the $update$ type, we will need to recalculate several segments with the new value of <Latex value="a[x]"> in mind. If you take a close look at the structure of the segments, index <Latex value="x"> appears exactly in one segment on every level. Since there is a total of <Latex value="log_2N"> levels, there will also be <Latex value="log_2N"> segments that need to be updated when <Latex value="a[x]"> changes.
The segments that need to be updated are the ones on the path from the root of the tree to the <Latex value="x">-th leaf, so again, a recursive call starting from the root will work well enough.
To better see the update on the tree, you can view the following animation, where element <Latex value="a[6]"> changes to <Latex value="9">:

<SegmentTreeUpdatePlayer N={16} array={[3, 2, 7, 1, 5, 4, 2, 8, 11, 31, 25, 2, 29, 1, 12, 3]} update={{x: 6, y: 9}}>

The implementation for the update is also mostly dependent on the `recalculate` function, the rest being usually the same.

The implementation for the update is also mostly dependent on the recalculate function, the rest being usually the same.




In [3]:
void update(int node, int left, int right, int x, int y) {
    if (left == right) { //we are in the xth leaf
        segm_tree[node] = y;
    } else {
        int middle = (left + right) / 2;
        if (x <= middle) { //we need to update the left son
            update(2 * node + 1, left, middle, x, y);
        } else {
            update(2 * node + 2, middle + 1, right, x, y);
        }
        //after updating said son, recalculate the current segment
        recalculate(node);
    }
}

...
update(0, 1, N, x, y);

SyntaxError: invalid syntax (<ipython-input-3-60a3bf5e2b42>, line 1)

At an operation of type query, we will split the query segment in several segments that we have the answer to in the segment tree. To see which segments are needed for the query, you can imagine that at first, we want the set of segments to contain all leaves from xx to yy. Then, while there are two segments that have the same father, remove them and add their father to the set instead. This is how a query segment looks like after the split (Query x=3, y=11x=3,y=11):


Implementing this, of course, will be the other way around: again, we start in the root and go down to the left son if the query segment and the left son segment have anything in common, and to the right son if the query segment and the right son segment have anything in common. When we reach a segment that is completely included in the query segment, return its solution instead.

In [4]:
int query(int node, int left, int right, int x, int y) {
    if (x <= left && right <= y) {
        //the segment of "node" is completely included in the query
        return segm_tree[node];
    } else {
        int answer = -Infinity;
        int middle = (left + right) / 2;
        if (x <= middle) {
            //the query segment and the left son segment have at least one element in common
            answer = max(answer, query(2 * node + 1, left, middle, x, y));
        }
        if (y >= middle + 1) {
            //the query segment and the right son segment have at least one element in common
            answer = max(answer, query(2 * node + 2, middle + 1, right, x, y));
        }
        //we would not have entered this function if (x, y) and (left, right) had nothing in common,
        //so there is no risk of answer returning -Infinity here, as either the left or the right son
        //would update it
        return answer;
    }
}


SyntaxError: invalid syntax (<ipython-input-4-4789c593ffc9>, line 1)

How fast is this actually?
As we can see in the implementation, the function might be called on both sons, so it's not very clear how many nodes actually get visited during a query. The query will go down only to one of the sons until at some point, it will split both ways. Let's take the sons of the node where the split takes place (call it <Latex value="S">) and call them <Latex value="L"> and <Latex value="R">. The picture will look something like this:

<StaticQueryProof >

As the query segment intersects with <Latex value="R"> as well, the query's right end is further right than <Latex value="L">'s right end. Also, since this is the first split, it means that the query is completely included in <Latex value="S">'s segment, so <Latex value="L">'s left end is further left than the query's.
With these observations, we can see that if by going down from <Latex value="L"> we were to split again, the *right* son of the split will always be completely included in the query, so it's solved in <Latex value="O(1)">. The same goes for *left* sons of splits down the right son of the first split. So after the first split, both sons will have at most <Latex value="O(log_2N)"> nodes visited. So in total, the time complexity is <Latex value="O(log_2N)"> per query.

#3 Lazy updates
Sometimes you need to update an entire segment, not just an element. Let's consider the initial problem, but with these operations instead:
- update: add <Latex value="v"> to all elements <Latex value="a[x], a[x+1]...a[y]">
- query: <Latex value="a[x]+a[x+1]+...+a[y]">
As the subtitle suggests, we will solve this problem in a /lazy/ fashion: an update will be applied to a node only when it is needed there. Each segment will have an associated $lazy$ value besides the needed info (in this case, the sum) that represents an update that needs to be applied to the $whole segment$. So an update will split its segment the same way the query does, and set the $lazy$ value on each segment reached to be equal to <Latex value="v">. Then, whenever we enter into a node, regardless whether through an update or a query, we will call a function, `propagate`, that updates the info in the current segment and passes the lazy value down to its sons. The code looks something like this:

In [5]:
struct Segment {
    int sum, lazy;
} segm_tree[2 * N];

void propagate(int node, int left, int right) {
    //calculate the sum of the current node by adding lazy to all elements from left to right
    segm_tree[node].sum += (right - left + 1) * segm_tree[node].lazy;
    //pass the lazy to node's sons if it is not a leaf
    if (left < right) {
        segm_tree[2 * node + 1].lazy += segm_tree[node].lazy;
        segm_tree[2 * node + 2].lazy += segm_tree[node].lazy;
    }
    //since the node is now up-to-date, lazy becomes 0
    segm_tree[node].lazy = 0;
}

void update(int node, int left, int right, int x, int y, int value) {
    propagate(node, left, right);
    if (x <= left && right <= y) {
        segm_tree[node].lazy += value;
    } else {
        int middle = (left + right) / 2;
        if (x <= middle) {
            update(2 * node + 1, left, middle, x, y, value);
        }
        if (y >= middle + 1) {
            update(2 * node + 2, middle + 1, right, x, y, value);
        }
        //we need to call propagate on the sons here, as we use them to update the current node
        //so they must be up-to-date
        propagate(2 * node + 1, left, middle);
        propagate(2 * node + 2, middle + 1, right);
        segm_tree[node].sum = segm_tree[2 * node + 1].sum + segm_tree[2 * node + 2].sum;
    }
}

int query(int node, int left, int right, int x, int y) {
    propagate(node, left, right);
    if (x <= left && right <= y) {
        return segm_tree[node].sum;
    } else {
        int sum = 0;
        int middle = (left + right) / 2;
        if (x <= middle) {
            sum += query(2 * node + 1, left, middle, x, y);
        }
        if (y >= middle + 1) {
            sum += query(2 * node + 2, middle + 1, right, x, y);
        }
        return sum;
    }
}


SyntaxError: invalid syntax (<ipython-input-5-6a868f916ccb>, line 1)